In [2]:
import sys
sys.path.insert(1, "C:\pdfminer-20140328")


In [3]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text



In [4]:
from os import path
from glob import glob 

def find_ext(dr, ext):
    return glob(path.join(dr,"*.{}".format(ext)))



In [5]:
liste = find_ext("./tazadata/","pdf")
for e in liste:
    text_file = open(str(e) + ".txt", "w")
    text_file.write(convert_pdf_to_txt(e))
    text_file.close()

In [39]:
# -*- coding: utf-8 -*-
import pandas as pd

liste = find_ext("./tazadata/","txt")

valuesTaza = []

for f in liste:
    lines = [line.rstrip('\n') for line in open(f)]
    #print str(f)[-29:]
    
    idxMalla = lines.index("Malla")
    idxItem = lines.index("Item")
    idxDefectos = lines.index("Defectos")
    idxCarac = lines.index("Características")
    idxTaza = lines.index("Tabla 5. Descripción de la taza")
    chapterIndexes = [idxMalla,idxItem,idxDefectos,idxCarac,idxTaza]
    
    #################################################
    
    #infos générales
    idxInfo = lines.index("A. DATOS DE LA MUESTRA")
    idxEndInfo = lines.index("B. ANÁLISIS FÍSICO")
    
    Tabla0Item = ["Nombre Caficultor","Municipio","Vereda","Finca","SICA","Grupo","Fecha Recpt","Fecha Analysis","No Orden"]
    Tabla0val= []
    
    decalage = 0
    
    nom = lines[idxInfo + 1]
    
    if(lines[idxInfo + 2].find("Nombre") != -1):
        decalage += 1
    
    municipio_vereda = lines[idxInfo + 2 + decalage] 
    
    
    
    '''finca_sica =  lines[idxInfo + 3]
    grupo =  lines[idxInfo + 4]
    
    fechaRec =  lines[idxInfo + 6]
    fechaAnalysis =  lines[idxInfo + 7]
    nbOrden =  lines[idxInfo + 8]'''
    
    
    # Obbligé car certain noms font 3 kilomètres et sont sur 2 lignes et c'est la manière la plus rapide (à coder)
    for i in range(idxInfo, idxEndInfo):
        l = lines[i]
        
        if(l.find("Nombre") != -1):
            Tabla0val.append(l[23:])
        
        if(l.find("Municipio") != -1):
            idxVereda = l.find("Vereda")
            idxVereda1 = idxVereda - 1
            idxVereda2 = idxVereda + 8
            
            Tabla0val.append(l[10:idxVereda1])
            Tabla0val.append(l[idxVereda2:])
        if(l.find("Finca") != -1):
            idxVereda = l.find("Código")
            idxVereda1 = idxVereda - 1
            idxVereda2 = idxVereda + 14
            
            Tabla0val.append(l[7:idxVereda1])
            Tabla0val.append(l[idxVereda2:])
        
        if(l.find("Grupo") != -1):
            Tabla0val.append(l[7:])
        
        if(l.find("Fecha de recibo") != -1):
            Tabla0val.append(l[17:])
            
        if(l.find("Fecha de análisis") != -1):
            Tabla0val.append(l[20:])
    
        if(l.find("Número de orden") != -1):
            Tabla0val.append(l[17:])
        
    
    
    
    
    #################################################
    
    # Granulomémtrie
    
    Tabla1Item = []
    Tabla1val= []
    
    for i in range(0,8):
        idxM = idxMalla + 2 + i
        idxVal = idxMalla + 12 + i
        
        colname = "Malla " + lines[idxM]
        Tabla1Item.append(colname)
        Tabla1val.append(lines[idxVal])
        #print("Malla: {}  value: {}".format(lines[idxM], lines[idxVal]))
    
    #print len(Tabla1Item)
    #print len(Tabla1val)
    
    #################################################
    
    # Caractéristiques physiques - éspaces vides aléatoires
    Tabla3Item = []
    Tabla3val = []
    
    for i in range(0,7):
        idxIt = idxItem + 4 + i
        Tabla3Item.append(lines[idxIt])
        #print("Item: {}".format(lines[idxIt]))
    
    idxItemFactor = lines.index("Factor de Rendimiento")
    
    s = ""
    for i in range(idxItemFactor,idxDefectos -1):
        # + 1 pour éviter le "factor de rendimiento"
        if str(lines[i+1]) == s:
            continue
        
        Tabla3val.append(lines[i+1])
        #print("Value: {}".format(lines[i+1]))
    
    # supprime le 1er (si c'est 8, le titre de la collone se met en première place)
    if len(Tabla3val) == 8:
        Tabla3val = Tabla3val[1:]
    
    #print len(Tabla3Item)
    #print len(Tabla3val)
    
    
    
    #################################################
    
    # Defectos
    
    Tabla2Item = []
    Tabla2val = []
    
    # index of the second (%) -> because right before the defectos values
    indices = [i for i, x in enumerate(lines) if x == "(%)"]
    indice = indices[1]
    
    for i in range(0, 20):
        idxIt = idxDefectos + 2 + i
        idxVal = indice + 1 + i
        
        Tabla2Item.append(lines[idxIt])
        Tabla2val.append(lines[idxVal])
        
    Tabla2val = ' '.join(Tabla2val).split()
    Tabla2Item = Tabla2Item[:-1]
    
    
    
    #print len(Tabla2Item)
    #print len(Tabla2val)
    
    #################################################
    
    # Caracteristicas
    
    Tabla4Item = []
    Tabla4val = []
    
    for i in range(0,10):
        idxIt = idxCarac + 4 + i
        idxVal = idxCarac + 17 + i
        
        Tabla4Item.append(lines[idxIt])
        Tabla4val.append(lines[idxVal])
    #print len(Tabla4Item)
    #print len(Tabla4val)
    
    #################################################
    
    # Description de la taza
    
    possibleValues = ["Limpia","Agrio","Vinagre","Fermento","Stinker","Metálico",
                      "Maderoso","Acre","Reposo","Terroso","Moho","Fenol",
                      "Químico","Quimico","Humo","Contaminado"]
    
    Tabla5Item = ["Taza 1","Taza 2","Taza 3","Taza 4","Taza 5"]
    Tabla5val = []
    
    found = False
    while found == False:
        idxTaza = idxTaza + 1
        #print("Line: {} : {}".format(idxTaza, lines[idxTaza]))
        if str(lines[idxTaza]) in possibleValues:
            found = True
    
    
    for i in range(0,5):
        idxVal = idxTaza  + i
        Tabla5val.append(lines[idxVal])

    filename = str(f)
    
    items = Tabla0Item + Tabla1Item + Tabla2Item + Tabla3Item + Tabla4Item + Tabla5Item
    values = Tabla0val + Tabla1val + Tabla2val + Tabla3val + Tabla4val + Tabla5val
    
    valuesTaza.append(values)
    

df = pd.DataFrame.from_records(valuesTaza, columns=items)
del df['Grupo']
del df['No Orden']



df['cedula'] = ""
df['variedad'] = ""
df['AlmendraTotal'] = ""
df['AlmendraSana'] = ""
df['Longitude'] = ""
df['Latitude'] = ""

df

,Nombre Caficultor,Municipio,Vereda,Finca,SICA,Fecha Recpt,Fecha Analysis,Malla 18,Malla 17,Malla 16,...,Taza 2,Taza 3,Taza 4,Taza 5,cedula,variedad,AlmendraTotal,AlmendraSana,Longitude,Latitude
0,Alvaro Mendez Bermudez,Balboa,La Cienaga,Palestina,06607500996,2011-11-24,2011-12-05,51.31,24.99,13.48,...,Limpia,Limpia,Limpia,Limpia,,,,,,
1,Facundo Quintero,Balboa,La Cienaga,La Albania,06607500981,2011-12-02,2011-12-05,42.1,33.86,15.4,...,Limpia,Limpia,Limpia,Limpia,,,,,,
2,Norberto Marquez Alvarez,Balboa,El Chuzcal,La Esperanza,06607501034,2011-11-24,2011-12-05,50.07,25.06,12.08,...,Limpia,Limpia,Limpia,Limpia,,,,,,
3,Aicardo Ortega,Balboa,Llanogrande,Altamira,06607500081,2011-11-24,2011-12-05,44.48,30.23,14.1,...,Limpia,Limpia,Limpia,Limpia,,,,,,
4,Amparo Vanegaz Trujillo,Balboa,Cocohondo,La Palma,06607500108,2011-11-24,2011-12-05,51.81,24.22,14.91,...,Limpia,Limpia,Limpia,Limpia,,,,,,
5,Norma Lorena Hernandez,Balboa,La Floresta,La Paiva,06607500309,2011-11-24,2011-12-05,37.16,30.1,18.48,...,Limpia,Limpia,Limpia,Limpia,,,,,,
6,Luis Enrique Laverde,Balboa,Tres Esquinas,La Milagrosa,06607500028,2011-11-24,2011-12-05,44.52,33.7,13.54,...,Agrio,Agrio,Agrio,None,,,,,,
7,Fernando Velez Franco,Balboa,Llanogrande,El Llano,06607500014,2011-11-24,2011-12-05,43.23,33.2,13.78,...,Limpia,Limpia,Limpia,Limpia,,,,,,
8,Luz Mery Garcia Largo,Balboa,Carminales,El Guadualito,06607500148,2011-11-24,2011-12-05,36.24,30.97,17.6,...,Limpia,Limpia,Limpia,Limpia,,,,,,
9,Jose William Cano,Balboa,Carminales,La Esperanza,06607500532,2011-12-02,2011-12-05,47.78,24.88,14.41,...,Limpia,Limpia,Limpia,None,,,,,,


In [40]:
# Reorder df
cols =  ['Nombre Caficultor', 'Municipio', 'Vereda', 'Finca', 'SICA', 'variedad', 'Fecha Recpt', 'Fecha Analysis', 
         'Malla 18', 'Malla 17', 'Malla 16', 'Malla 15', 'Malla 14', 'Malla 13', 'Malla 12', 'Malla 0', 
         'Negros y vinagres', 'Broca', 'Broca de punto', 'Veteado', 'Mordido', 'Inmaduro', 'Flojo', 
         'Sobresecado', 'Arrugado', 'Aplastado', 'Cristalizado', 'Reposado', 'Granizo', 'Conchas', 
         'Partido', 'Ambar', 'Defectos totales', 'Quaker (N\xc2\xb0 Granos)', 'Fluorescentes (N\xc2\xb0 Granos)', 
         'Pergamino', 'Almendra', 'UV', 'Olor', 'Humedad (%)', 'Merma (%)', 'Factor de Rendimiento', 
         'Aroma-Fragancia', 'Acidez', 'Cuerpo', 'Sabor', 'Sabor residual', 'Dulzor', 'Uniformidad', 'Balance', 
         'Taza limpia', 'Puntaje catador', 'Taza 1', 'Taza 2', 'Taza 3', 'Taza 4', 'Taza 5', 'cedula', 'variedad', 
         'AlmendraTotal', 'AlmendraSana']
colsOrdered = ['Nombre Caficultor', 'cedula', 'Municipio', 'Vereda', 'Finca', 'SICA','Longitude','Latitude','variedad', 'Fecha Recpt', 'Fecha Analysis', 
         'Malla 18', 'Malla 17', 'Malla 16', 'Malla 15', 'Malla 14', 'Malla 13', 'Malla 12', 'Malla 0', 
         'Negros y vinagres', 'Broca', 'Broca de punto', 'Veteado', 'Mordido', 'Inmaduro', 'Flojo', 
         'Sobresecado', 'Arrugado', 'Aplastado', 'Cristalizado', 'Reposado', 'Granizo', 'Conchas', 
         'Partido', 'Ambar', 'Defectos totales', 'Quaker (N\xc2\xb0 Granos)', 'Fluorescentes (N\xc2\xb0 Granos)', 
         'Pergamino', 'Almendra','AlmendraTotal', 'AlmendraSana', 'UV', 'Olor', 'Humedad (%)', 'Merma (%)', 'Factor de Rendimiento', 
         'Aroma-Fragancia', 'Acidez', 'Cuerpo', 'Sabor', 'Sabor residual', 'Dulzor', 'Uniformidad', 'Balance', 
         'Taza limpia', 'Puntaje catador', 'Taza 1', 'Taza 2', 'Taza 3', 'Taza 4', 'Taza 5']




df = df[colsOrdered]

df

,Nombre Caficultor,cedula,Municipio,Vereda,Finca,SICA,Longitude,Latitude,variedad,Fecha Recpt,...,Dulzor,Uniformidad,Balance,Taza limpia,Puntaje catador,Taza 1,Taza 2,Taza 3,Taza 4,Taza 5
0,Alvaro Mendez Bermudez,,Balboa,La Cienaga,Palestina,06607500996,,,,2011-11-24,...,10.0,10,7.25,10,7.5,Limpia,Limpia,Limpia,Limpia,Limpia
1,Facundo Quintero,,Balboa,La Cienaga,La Albania,06607500981,,,,2011-12-02,...,10.0,10,7.0,10,7.0,Limpia,Limpia,Limpia,Limpia,Limpia
2,Norberto Marquez Alvarez,,Balboa,El Chuzcal,La Esperanza,06607501034,,,,2011-11-24,...,10.0,10,7.0,10,6.5,Limpia,Limpia,Limpia,Limpia,Limpia
3,Aicardo Ortega,,Balboa,Llanogrande,Altamira,06607500081,,,,2011-11-24,...,10.0,10,7.0,10,7.5,Limpia,Limpia,Limpia,Limpia,Limpia
4,Amparo Vanegaz Trujillo,,Balboa,Cocohondo,La Palma,06607500108,,,,2011-11-24,...,10.0,10,6.25,10,6.25,Limpia,Limpia,Limpia,Limpia,Limpia
5,Norma Lorena Hernandez,,Balboa,La Floresta,La Paiva,06607500309,,,,2011-11-24,...,10.0,10,7.0,10,7.25,Limpia,Limpia,Limpia,Limpia,Limpia
6,Luis Enrique Laverde,,Balboa,Tres Esquinas,La Milagrosa,06607500028,,,,2011-11-24,...,0,0.0,0,0.0,Agrio,Agrio,Agrio,Agrio,Agrio,None
7,Fernando Velez Franco,,Balboa,Llanogrande,El Llano,06607500014,,,,2011-11-24,...,10.0,10,6.5,10,6.5,Limpia,Limpia,Limpia,Limpia,Limpia
8,Luz Mery Garcia Largo,,Balboa,Carminales,El Guadualito,06607500148,,,,2011-11-24,...,10.0,10,6.25,10,6.0,Limpia,Limpia,Limpia,Limpia,Limpia
9,Jose William Cano,,Balboa,Carminales,La Esperanza,06607500532,,,,2011-12-02,...,10,7.5,10,7.5,Limpia,Limpia,Limpia,Limpia,Limpia,None


In [41]:
with pd.option_context('display.max_rows', None, 'display.max_columns', 49):
    print(df)

               Nombre Caficultor cedula             Municipio  \
0         Alvaro Mendez Bermudez                       Balboa   
1               Facundo Quintero                       Balboa   
2       Norberto Marquez Alvarez                       Balboa   
3                 Aicardo Ortega                       Balboa   
4        Amparo Vanegaz Trujillo                       Balboa   
5         Norma Lorena Hernandez                       Balboa   
6           Luis Enrique Laverde                       Balboa   
7          Fernando Velez Franco                       Balboa   
8          Luz Mery Garcia Largo                       Balboa   
9              Jose William Cano                       Balboa   
10             Jose Climaco Diaz                       Balboa   
11         Gladys Noreña Tabares                       Balboa   
12          Jairo Antonio Noreña                       Balboa   
13             Jorge Marin Velez                       Balboa   
14          Luis Alberto 

In [44]:
df.to_csv("TazaDatosFromPDF.csv", sep=',', encoding='utf-8')

In [45]:
df.columns
df.shape

(617, 62)

In [46]:
df.T[6]


Nombre Caficultor               Luis Enrique Laverde
cedula                                              
Municipio                                     Balboa
Vereda                                 Tres Esquinas
Finca                                   La Milagrosa
SICA                                     06607500028
Longitude                                           
Latitude                                            
variedad                                            
Fecha Recpt                               2011-11-24
Fecha Analysis                            2011-12-05
Malla 18                                       44.52
Malla 17                                        33.7
Malla 16                                       13.54
Malla 15                                        4.99
Malla 14                                        2.37
Malla 13                                        0.74
Malla 12                                         0.1
Malla 0                                       